In [1]:
import torch
torch.cuda.is_available()

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/rebs/dev/gpt2/.env/lib/python3.9/site-packages/traitlets/traitlets.py", line 632, in get
    value = obj._trait_values[self.name]
KeyError: '_control_lock'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/rebs/dev/gpt2/.env/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/home/rebs/dev/gpt2/.env/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 301, in dispatch_control
    async with self._control_lock:
  File "/home/rebs/dev/gpt2/.env/lib/python3.9/site-packages/traitlets/traitlets.py", line 687, in __get__
    return t.cast(G, self.get(obj, cls))  # the G should encode the Optional
  File "/home/rebs/dev/gpt2/.env/lib/python3.9/site-packages/traitlets/traitlets.py", line 649, in get
    value = self._validate(obj, default)
  File "/home/rebs

True

In [2]:
from dataclasses import dataclass
import torch.nn as nn
from torch.nn import functional as F
import math

In [3]:
import tiktoken

Sources
- https://www.youtube.com/watch?v=l8pRSuU81PU&t=1839s&ab_channel=AndrejKarpathy

In [4]:
@dataclass
class GPTConfig:
    context_size: int = 1024
    vocab_size: int = 50257
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768

In [68]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.c_proj.GPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd

        self.register_buffer('bias', torch.tril(torch.ones(config.context_size, config.context_size))
                                    .view(1, 1, config.context_size, config.context_size))

    def forward(self, x):
        b,t,c = x.size()
        qkv = self.c_attn(x)
        q,k,v = qkv.split(self.n_embd, dim=2)
        q = q.view(b,t, self.n_head, c // self.n_head).transpose(1,2)
        k = k.view(b,t, self.n_head, c // self.n_head).transpose(1,2)
        v = v.view(b,t, self.n_head, c // self.n_head).transpose(1,2)

        #att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        #att = att.masked_fill(self.bias[:,:,:t,:t] == 0, float('-inf'))
        #att = F.softmax(att, dim=-1)
        #y = att @ v
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        y = y.transpose(1,2).contiguous().view(b,t,c)
        y = self.c_proj(y)
        return y

In [70]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd)
        self.c_proj.GPT_SCALE_INIT = 1

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

In [7]:
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = MultiHeadAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [71]:
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), # weight token embedding
            wpe = nn.Embedding(config.context_size, config.n_embd), # weight position embedding
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight

        # initialize parameters
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module,'GPT_SCALE_INIT'):
                std *= (2 * self.config.n_layer) ** -0.5 # number of residual adds * number 
                # of layers 1/sqrt to clip scaling for each projection layer
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=std)

    def forward(self, idx, targets=None):
        b, t = idx.size()
        assert t <= self.config.context_size, f"sequence too long"
        pos = torch.arange(0, t, dtype=torch.long, device=idx.device)
        pos_emb = self.transformer.wpe(pos)
        tok_emb = self.transformer.wte(idx)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        return logits, loss

    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['context_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [72]:
#tokens = enc.encode("Hello, I'm a language model,")
#tokens = torch.tensor(tokens, dtype=torch.long)
#tokens = tokens.unsqueeze(0).repeat(5, 1)
#x = tokens.to('cuda')

In [73]:
#torch.save(model.state_dict(), './data/hugging_face_weights')

In [74]:
#model2 = GPT(GPTConfig())
#model2.load_state_dict(torch.load('./data/hugging_face_weights', weights_only=True))
#model2.eval()
#model2.to('cuda')

In [75]:
class DataLoaderLite:
    def __init__(self, b, t):
        self.b = b
        self.t = t

        with open('./data/tiny_shakespeare.txt','r') as f: text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f'loaded {len(self.tokens)} tokens')
        print(f' 1 epoch = {len(self.tokens) // (b*t)} batches')

        self.current_position = 0

    def next_batch(self):
        b, t = self.b, self.t
        buf = self.tokens[self.current_position : self.current_position+b*t+1]
        x = (buf[:-1]).view(b, t)
        y = (buf[1:]).view(b, t)
        self.current_position += b * t
        if self.current_position + (b * t + 1) > len(self.tokens):
            self.current_position = 0
        return x, y

In [76]:
train_loader = DataLoaderLite(4, 32)

loaded 338024 tokens
 1 epoch = 2640 batches


In [77]:
model = GPT(GPTConfig())
model.to('cuda')

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): MultiHeadAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [78]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

default loss should be negative log likeliehood aka -ln(1/50257)

In [79]:
for i in range(50):
    x, y = train_loader.next_batch()
    x, y = x.to('cuda'), y.to('cuda')
    optimizer.zero_grad()
    logits, loss = model(x, y)
    loss.backward()
    optimizer.step()
    print(f'step {i}, loss: {loss.item()}')

step 0, loss: 10.956216812133789
step 1, loss: 9.832077026367188
step 2, loss: 8.987883567810059
step 3, loss: 9.051115036010742
step 4, loss: 8.738351821899414
step 5, loss: 8.28447151184082
step 6, loss: 8.928132057189941
step 7, loss: 8.827741622924805
step 8, loss: 8.060985565185547
step 9, loss: 7.874083042144775
step 10, loss: 8.306632995605469
step 11, loss: 7.3382415771484375
step 12, loss: 7.704967021942139
step 13, loss: 7.397910118103027
step 14, loss: 7.546712875366211
step 15, loss: 7.327558517456055
step 16, loss: 7.449942588806152
step 17, loss: 8.215373039245605
step 18, loss: 7.140617370605469
step 19, loss: 7.774016857147217
step 20, loss: 7.4927520751953125
step 21, loss: 7.800985336303711
step 22, loss: 6.53911828994751
step 23, loss: 6.895722389221191
step 24, loss: 6.895887851715088
step 25, loss: 6.736913681030273
step 26, loss: 6.8208909034729
step 27, loss: 7.683427810668945
step 28, loss: 7.249087333679199
step 29, loss: 7.107006549835205
step 30, loss: 6.9801